In [1]:
import iparser
import pandas as pd
import importlib
import pickle
import spacy
import tqdm
import utils 
from spacy.lang.en import English
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')

## Топ профессий:

 

* Java разработчик/Software engineer
* Front-end разработчик/Web разработчик
* Продуктовый дизайнер
* Системный аналитик
* Архитектор систем
* Agile coach (под вопросом?)
* Data Scientist
* Data engineer
* Бизнес-аналитик
* Финансовый аналитик
* Кредитный аналитик
* Юрист (корпоративный, судебный)
* Маркетолог
 

 

# Используемый Подход 
* Парсер с определенной периодичностью запускается и соберает вакансии в базу
* Для анализа данные выгружаются из базы в датафрейм 


In [2]:
importlib.reload(iparser)
importlib.reload(utils)
pdict = utils.patterns_dict

In [3]:
vacancy_db = "vacancies.sqlite3"
import db_handler 
importlib.reload(db_handler)
db = db_handler.Db_handler(vacancy_db)
db.create_tab()

In [4]:
# for vaca in v.vacancies:
#     db.addvacancy(vaca)

# v_ids = [x[0] for x in db.show_vacancy_ids()]

cols = ['vacancy_id','description','profession','country','city','reqs','skills','date']
vtab = pd.DataFrame(db.show_vacancies(), columns=cols)

In [5]:
vtab.shape

(117495, 8)

In [6]:
vtab.head()

,vacancy_id,description,profession,country,city,reqs,skills,date
0,/rc/clk?jk=8fb6adb20e0cc7c0&fccid=fe2d21eef233...,7+ years of relevant engineering experience3+ ...,Agile+coach,us,New+York,,,05-06-2020
1,/rc/clk?jk=66526d69241caf2c&fccid=48638adc9594...,"Innovation, collaboration, and success: at OnD...",Agile+coach,us,New+York,,,05-06-2020
2,/rc/clk?jk=9d8e229307c5f80e&fccid=ec00141efcac...,"At the NBA, we’re passionate about growing and...",Agile+coach,us,New+York,,,05-06-2020
3,/rc/clk?jk=42f82312c8e47d82&fccid=319b94ac5d94...,CLEAR's mission is to strengthen security and ...,Agile+coach,us,New+York,,,05-06-2020
4,/rc/clk?jk=437873289bb297a9&fccid=8009d67e7070...,Agile Coach Rock Star - hiring globally - (SO0...,Agile+coach,us,New+York,,,05-06-2020


In [11]:
# vtab.profession.unique()

In [8]:
# vtab[vtab.profession == 'Data+Scientist'].city.unique()

In [12]:
vtab[vtab.profession == 'Data+Scientist'].vacancy_id.nunique()

3641

In [13]:
vtab.groupby(['profession'])['vacancy_id'].count().reset_index()

,profession,vacancy_id
0,Agile+coach,1874
1,Business-analyst,3804
2,Corporate+lawyer,1017
3,Credit+analyst,5899
4,Data+Scientist,3641
5,Data+engineer,13880
6,Financial+analyst,2771
7,Front-end+Developer,5628
8,Java+Developer,8073
9,Judicial+Lawyer,91


In [10]:
vtab[vtab.profession == 'Data+Scientist'].groupby(['profession','city'])['vacancy_id'].count().reset_index()

,profession,city,vacancy_id
0,Data+Scientist,Belfast,15
1,Data+Scientist,Birmingham,18
2,Data+Scientist,Bristol,14
3,Data+Scientist,California,1191
4,Data+Scientist,Dallas,91
5,Data+Scientist,Edinburgh,19
6,Data+Scientist,Georgia,104
7,Data+Scientist,Glasgow,5
8,Data+Scientist,Hawaii,16
9,Data+Scientist,Idaho,5


In [14]:
vacs_desc = vtab[vtab.profession == 'Data+Scientist']['description'].tolist()

In [35]:
vacs_desc[11]

'Brinks Home Security™ is a proven leader in the smart home technology and residential security industries, providing cutting-edge products and alarm monitoring services to more than one million customers throughout North America. We are currently seeking a motivated and dynamic Senior Data Scientist.\nOur brand promise delivers on 4 core principles:\nIntegrity in Action: Be True, Be Responsive, Be Accountable\nSense of Urgency: Every Moment Matters\nRelentless Discipline: Rigor, Precision and Excellence in Every Single Action\nResults Matter: Account to Customers, Employees & Stakeholders\n\nJoin our team of trusted security advisors and providers and help us create more secure smart home customers.\nSummary:\nBrinks Home Security’s Business Intelligence team is rapidly growing and is seeking a Senior Data Scientist to advise on and implement machine learning models to improve customer acquisition and customer retention efforts across the organization.\nEssential Duties and Responsibi

In [16]:
nlp = spacy.load('en_core_web_lg')
matcher = Matcher(nlp.vocab)
for k, el in pdict.items():
    matcher.add(k, None, el)

In [18]:
def requirements_extractor(doc, matcher=matcher):
    endpos = None
    matches = matcher(doc)
        
    if len(matches) > 0 :
        startpos = matches[0][1]
        for el in doc[startpos:]:
            if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
#                 print(doc[el.i+1], el.i+1)
                endpos = el.i + 1
                break
    if endpos:
        return doc[startpos:endpos]
    else:
        return None
    

In [25]:
s = vacs_desc[6]
doc = nlp(s)
res = requirements_extractor(doc)

In [24]:
res

In [26]:
reqs = []
for s in tqdm.notebook.tqdm(vacs_desc):
    doc = nlp(s)
    res = requirements_extractor(doc)
    if res:
        if len(res) > 4:
#             print("_"*10)
#             print(res)
#             print(len(res))
            reqs.append(res.text)

In [27]:
len(reqs)

418

In [30]:
import re
s = reqs[43]
def hanmade_tokenizer(s):
    res = re.split(r"\n|;|,.",s)
    res = [x.strip() for x in res]
    res = [x for x in res if x not in ['']]
    return res

In [31]:
# re.split(r"\n|;|,.",s)
s = reqs[46]
hanmade_tokenizer(s)

['ideal candidate will have a strong grasp of software development fundamentals',
 'along with an interest in learning more about both new cutting-edge technologies and the Chemistry',
 'Biology',
 'and Physics problems that our software helps to solve.']

In [26]:
biglist = []
for s in reqs:
    res = hanmade_tokenizer(s)
    res = [x for x in res if x not in ['',' ']]
    biglist.extend(res)

In [27]:
txtdf = pd.DataFrame(biglist,columns=['text'])

In [28]:
import spacy #load spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'tagger', 'ner'])
# stops = stopwords.words("english")
stops = spacy.lang.en.stop_words.STOP_WORDS

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)


txtdf['norm_text'] = txtdf['text'].apply(normalize, lowercase=True, remove_stopwords=True)

In [29]:
norm_text = txtdf['norm_text'].tolist()
nlp = spacy.load('en_core_web_sm')
docs = []
for s in tqdm.tqdm_notebook(norm_text):
    docs.append(nlp(s))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [30]:
docs[10].vector.shape

(96,)

In [31]:
len(list(set(docs)))x

5210

In [32]:
nlp = spacy.load("en_core_web_lg")  # make sure to use larger model!

In [61]:
from sklearn.metrics.pairwise import cosine_similarity
cs = {}
bl = []
docs1 = []
docs2 = []
simsv = []
for doc1 in tqdm.tqdm_notebook(docs[:1000]):
    tres = {}
    for doc2 in docs[:1000]:
        try:
            r = cosine_similarity(doc1.vector.reshape(1, -1), doc2.vector.reshape(1, -1))
            if r > 0.85
                tres[doc2.text] = r[0][0]
                docs1.append(doc1)
                docs2.append(docs2)
                simsv.append(r[0][0])
        except ValueError :
            bl.append(doc2.text)
    
    cs[doc1] = tres

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [ ]:
simdf = pd.DataFrame({'itm1': docs1, 'itm2': docs2, 'score':simsv})

bring establish sale method sale process

In [38]:
# with open('sims1k.pkl', 'wb') as f:
#     pickle.dump(cs,f)

In [ ]:
cs_new = {}
for k,v in cs.items():
    for kk,vv in v.items():
        tres = 
    

In [58]:
ks = list(cs.keys())[4]

In [59]:
ks
cs[ks]

{'ideal candidate 5 + year internal external consult datum engineer experience': 0.6898728,
 'specifically develope structure datum use analysis pipeline . time': 0.73864394,
 'demonstrate concrete evidence success follow 5 category key success role .': 0.74880385,
 'qualification': 0.525105,
 'bring establish sale method sale process': 1.0,
 'o develop organize differentiate market activity': 0.6894925,
 'o develop overview material support initial meeting / conversation': 0.8163768,
 'o lead preparation formal sale meeting oral qualify opportunity': 0.82511085,
 'o provide support core account cres need critical i&e opportunity': 0.68343294,
 'o identify opportunity ( sole source / bid ) bring business ( functional ) partner': 0.6198737,
 'evaluate opportunity alignment client strategy': 0.7287147,
 'o identify align appropriate firm resource pursue': 0.6335733,
 'win': 0.4055928,
 'manage opportunity': 0.7181171,
 'o lead pursuit process': 0.66413814,
 'rfp response': 0.6209146,
 'e

In [50]:
cs[ks]['product - drive technology team .'][0][0]

0.4051752

{ideal candidate 5 + year internal external consult datum engineer experience: {'product - drive technology team .': array([[0.6457797]], dtype=float32)},
 specifically develope structure datum use analysis pipeline . time: {'product - drive technology team .': array([[0.6937572]], dtype=float32)},
 demonstrate concrete evidence success follow 5 category key success role .: {'product - drive technology team .': array([[0.71832186]], dtype=float32)},
 qualification: {'product - drive technology team .': array([[0.4051752]], dtype=float32)},
 bring establish sale method sale process: {'product - drive technology team .': array([[0.6562269]], dtype=float32)},
 o develop organize differentiate market activity: {'product - drive technology team .': array([[0.57690036]], dtype=float32)},
 o develop overview material support initial meeting / conversation: {'product - drive technology team .': array([[0.66611284]], dtype=float32)},
 o lead preparation formal sale meeting oral qualify opportun

In [200]:

cosine_similarity(docs[1].vector.reshape(1, -1), docs[2].vector.reshape(1, -1))

array([[0.63597155]], dtype=float32)

array([[ 1.99165702e+00, -1.25236297e+00, -9.37911034e-01,
         6.58678293e-01, -9.05168653e-02,  1.07294381e-01,
         1.22539759e+00,  8.40529442e-01,  1.20618784e+00,
         1.96703482e+00,  3.16312003e+00, -1.70996904e-01,
         1.88429630e+00, -2.60295391e+00,  7.86821365e-01,
        -1.97734535e+00,  3.06851804e-01,  1.92688203e+00,
        -2.04203176e+00, -3.25758249e-01,  2.60205102e+00,
         8.58381838e-02, -5.75799465e-01, -7.81261802e-01,
        -2.31686926e+00, -1.53583968e+00, -5.89183509e-01,
        -2.64788318e+00,  2.89194918e+00, -3.24019361e+00,
         3.71261454e+00,  2.01283693e-01,  5.84064245e-01,
         7.13174164e-01,  1.84596241e-01, -1.82998288e+00,
         4.05095387e+00, -2.57433265e-01, -3.62787318e+00,
         5.13861835e-01,  4.78788424e+00, -4.78212059e-01,
         4.21033978e-01, -2.67334509e+00,  6.29396975e-01,
        -1.30750227e+00, -9.21405554e-02, -5.93005478e-01,
        -2.18722773e+00, -2.55334377e-03, -9.90116715e-0

In [154]:
tp = TextPreprocessor()

In [155]:
tp

TextPreprocessor(n_jobs=1, user_abbrevs={}, variety='BrE')

In [152]:
tp.fit(rawskills)

AttributeError: 'tuple' object has no attribute 'fit'

In [166]:
requirements_extractor(doc)

In [254]:
s = vacancy_dict['Data Scientist'][30]
doc = nlp(s)
# doc

In [121]:
matcher = Matcher(nlp.vocab)

pattern = [
    {'LEMMA':'requirement'},
    {'LEMMA':'skills'},
    {'LEMMA':'experience'},
]

In [122]:
for el in pdict.values():
    matcher.add('REQ_PATTERN', None, el)

In [150]:
# doc[502:]

In [116]:
c = 0
for el in doc[502:]:
    if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
        print(doc[el.i+1], el.i+1)
        c = el.i + 1
        break


Accountabilities 600


In [124]:
doc[matches[0][1]:c]

Requirements:
A Finance/Economics/Business/Mathematics graduate
A demonstrated interest and knowledge in the financial markets
A proven record of outstanding achievement in academic and extracurricular activities
An independent thinker and decision maker who is able to contribute effectively to the team's success
Superior interpersonal and analytical skills
Ability to meet the challenges presented by a stressful, constantly changing work environment.
Ideally available to start asap – individual will be expected to go to Toronto for training 13th August for 2 wks.






In [123]:
matcher = Matcher(nlp.vocab)
for el in pdict.values():
    matcher.add('REQ_PATTERN', None, el)

In [98]:
def requirements_extractor(doc, matcher=matcher):
    endpos = None
    matches = matcher(doc)
        
    if len(matches) > 0 :
        startpos = matches[0][1]
        for el in doc[startpos:]:
            if len(el.text.split("\n")) > 2 and (el.i) < len(doc):
#                 print(doc[el.i+1], el.i+1)
                endpos = el.i + 1
                break
    if endpos:
        return doc[startpos:endpos]
    else:
        return None
    

In [99]:
reqs = []
for s in tqdm.notebook.tqdm(vacs_desc):
    doc = nlp(s)
    res = requirements_extractor(doc)
    if res:
        if len(res) > 4:
#             print("_"*10)
#             print(res)
#             print(len(res))
            reqs.append(res)
print(f"Удалось извлечь потенциальных скилов {len(reqs)} шт.")

ValueError: [E155] The pipeline needs to include a tagger in order to use Matcher or PhraseMatcher with the attributes POS, TAG, or LEMMA. Try using nlp() instead of nlp.make_doc() or list(nlp.pipe()) instead of list(nlp.tokenizer.pipe()).

In [161]:
bv = vacs_dict['Agile coach'][2]

In [162]:
docbv = nlp(bv)

In [163]:
docbv

Global Markets (Credit Products), High Yield Sector Strategist, VP, London, #152732
United Kingdom-London-London | Full-time (FT) | Global Markets | Job ID 152732
Credit Suisse is a leading global wealth manager with strong investment banking capabilities. Headquartered in Zurich, Switzerland, we have a global reach with operations in about 50 countries and employ more than 45,000 people from over 150 different nations. Embodying entrepreneurial spirit, Credit Suisse delivers holistic financial solutions to our clients, including innovative products and specially tailored advice. Striving for quality and excellence in our work, we recognize and reward extraordinary performance among our employees, provide wide-ranging training and development opportunities, and benefit from a diverse range of perspectives to create value for our clients, shareholders and communities. We are Credit Suisse.
We Offer

The Credit Suisse leverage finance trading desk is looking for a desk analyst at the VP/